In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import torch
import torch.autograd as autograd
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim
import os
from glob import glob
from sklearn.metrics import roc_auc_score
import bcolz
from sklearn.cross_validation import train_test_split
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]
cwd = os.getcwd()
data_dir = os.path.join(cwd, 'data')
im_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
cwd

'/home/ubuntu/nbs/invasive_species'

## Flip Test Picture

My model below threw errors when making predictions on the test set, and after some digging I found that one test image is in the wrong orientation. So in the code below I flip it to match all the other images. 

In [4]:
% cd $test_dir

/home/ubuntu/nbs/invasive_species/data/test


In [5]:
im = Image.open('1068.jpg')

In [6]:
im.size

(866, 1154)

In [7]:
rot = im.transpose(Image.ROTATE_270) 

In [8]:
rot.size

(1154, 866)

In [9]:
rot.save('1068.jpg')

In [10]:
check = Image.open('1068.jpg')

In [11]:
check.size

(1154, 866)

## Train Val Split

In [12]:
% cd $data_dir

/home/ubuntu/nbs/invasive_species/data


Use the train labels file to split the training images into train and validation sets. 

In [19]:
labels = pd.read_csv(data_dir + '/train_labels.csv')
print(len(labels))
labels.head()

2295


,name,invasive
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1


In [22]:
print(labels.invasive.sum())
1448.0 / 2295.0

1448


0.6309368191721133

As we can see, 63% of the images are in the positive class. This doesn't seem super imbalanced, but we'll still use a stratified split.

In [23]:
tr_idx, val_idx = train_test_split(np.arange(len(labels)), stratify=labels.invasive.values, test_size = 0.15)

In [24]:
tr_idx.shape

(1950,)

In [25]:
val_idx.shape

(345,)

In [26]:
assert 1950 + 345 == len(labels)

In [27]:
% cd $im_dir

/home/ubuntu/nbs/invasive_species/data/train


Make positive and negative image folders for both the train and validation sets, then move the images to the appropriate folder.

In [28]:
% mkdir tr
% mkdir val

In [29]:
% mkdir tr/0
% mkdir tr/1
% mkdir val/0
% mkdir val/1

In [30]:
for idx in tr_idx:
    name = str(idx+1) + '.jpg'
    label = labels[labels.name == idx+1].invasive
    os.rename(name, os.path.join('tr', str(label.values[0]), name))

In [31]:
for idx in val_idx:
    name = str(idx+1) + '.jpg'
    label = labels[labels.name == idx+1].invasive
    os.rename(name, os.path.join('val', str(label.values[0]), name))

Move test images into a 'wrap' folder to make it easier to load them using pytorch's data loader

In [41]:
cd $test_dir

/home/ubuntu/nbs/invasive_species/data/test


In [ ]:
% mkdir wrap
% mv *.jpg wrap/

Check that we've moved the images correctly.

In [32]:
os.listdir('tr')

['1', '0']

In [33]:
os.listdir('val')

['1', '0']

In [44]:
% cd $im_dir/tr/0
g = glob('*.jpg')
print(len(g))

/home/ubuntu/nbs/invasive_species/data/train/tr/0
720


In [45]:
% cd $im_dir/tr/1
g = glob('*.jpg')
print(len(g))

/home/ubuntu/nbs/invasive_species/data/train/tr/1
1230


In [46]:
1230.0 / (1230 + 720) * 1.0

0.6307692307692307

In [47]:
% cd $im_dir/val/0
g = glob('*.jpg')
print(len(g))

/home/ubuntu/nbs/invasive_species/data/train/val/0
127


In [48]:
% cd $im_dir/val/1
g = glob('*.jpg')
print(len(g))

/home/ubuntu/nbs/invasive_species/data/train/val/1
218


In [49]:
218.0 / (127 + 218) * 1.0

0.6318840579710145

## Build Model
We're going to finetune a pretrained VGG-16 model, a task for which I found the following references helpful: 
- https://medium.com/towards-data-science/transfer-learning-using-pytorch-part-2-9c5b18e15551
- https://discuss.pytorch.org/t/how-to-perform-finetuning-in-pytorch/419/16




#### Precompute the convolutional features
First, we'll precompute the output of the convolutional layers of the VGG-16 model.


The following code set ups the infrastructure to load the image data, loads the pretrained model, and makes sure we're running on the GPU.

In [105]:
vgg_transform = transforms.Compose([
    transforms.Scale((224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

vgg_ds_tr = torchvision.datasets.ImageFolder(root=im_dir + '/tr', transform=vgg_transform)
vgg_ds_val = torchvision.datasets.ImageFolder(root=im_dir + '/val', transform=vgg_transform)
vgg_ds_test = torchvision.datasets.ImageFolder(root=test_dir, transform=vgg_transform)
vgg_tr_loader = torch.utils.data.DataLoader(vgg_ds_tr, batch_size=2, shuffle=False)
vgg_val_loader = torch.utils.data.DataLoader(vgg_ds_val, batch_size=2, shuffle=False)
vgg_test_loader = torch.utils.data.DataLoader(vgg_ds_test, batch_size=1, shuffle=False)

model_vgg = models.vgg16(pretrained=True)

for param in model_vgg.parameters():
    param.requires_grad = False


def preComputeConvFeat(dataset):
    conv_features = []
    labels_list = []
    for data in dataset:
        inputs,labels = data
        inputs , labels = autograd.Variable(inputs.cuda()),autograd.Variable(labels.cuda())
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    conv_features = np.concatenate([[feat] for feat in conv_features])
        
    return (conv_features,labels_list)


model_vgg.cuda()

VGG (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU (inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU (inplace)
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU (inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU (inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU (inplace)
    (16): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), pa

Now we can compute the output of the convolutional layers.

In [106]:
tr_conv_feats, tr_labels = preComputeConvFeat(vgg_tr_loader)

In [107]:
tr_conv_feats.shape

(1950, 512, 7, 9)

In [108]:
% cd $cwd

/home/ubuntu/nbs/invasive_species


In [109]:
save_array('tr_conv_feats.bc', tr_conv_feats)

In [110]:
tr_conv_feats = load_array('tr_conv_feats.bc')

In [111]:
val_conv_feats, val_labels = preComputeConvFeat(vgg_val_loader)

In [112]:
val_conv_feats.shape

(345, 512, 7, 9)

In [113]:
save_array('val_conv_feats.bc', val_conv_feats)

In [114]:
val_conv_feats = load_array('val_conv_feats.bc')

In [115]:
len(tr_labels)

1950

In [116]:
len(val_labels)

345

In [117]:
save_array('tr_labels.bc', tr_labels)
save_array('val_labels.bc', val_labels)

In [118]:
tr_labels = load_array('tr_labels.bc')
val_labels = load_array('val_labels.bc')

In [119]:
test_conv_feats, test_labels = preComputeConvFeat(vgg_test_loader)

In [120]:
test_conv_feats.shape

(1531, 512, 7, 9)

In [121]:
save_array('test_conv_feats.bc', test_conv_feats)

In [122]:
test_conv_feats = load_array('test_conv_feats.bc')

In [123]:
test_imgs = [f[0].split('/')[-1] for f in vgg_ds_test.imgs]

In [124]:
print(len(test_imgs))
test_imgs[0]

1531


'779.jpg'

In [125]:
save_array('test_filenames.bc', test_imgs)

In [126]:
test_imgs = load_array('test_filenames.bc')

#### Train the classifier

Now we can use the outputs from the convolutional layers as input to a small fully connected network that can produce the probability of each image containing the invasive species.

In [213]:
classifier = nn.Sequential(nn.Linear(32256, 4096), nn.ReLU(), nn.Dropout2d(p=0.75), nn.Linear(4096, 2), nn.LogSoftmax())
classifier.train()

Sequential (
  (0): Linear (32256 -> 4096)
  (1): ReLU ()
  (2): Dropout2d (p=0.75)
  (3): Linear (4096 -> 2)
  (4): LogSoftmax ()
)

In [214]:
for p in classifier.parameters():
    print(p.size())
    print(p.requires_grad)

torch.Size([4096, 32256])
True
torch.Size([4096])
True
torch.Size([2, 4096])
True
torch.Size([2])
True


Set up the input for our model, our loss function and our optimizer.

In [215]:
tr_tense = torch.Tensor(tr_conv_feats.reshape(tr_conv_feats.shape[0], -1))
tr_tense.size()

torch.Size([1950, 32256])

In [216]:
val_tense = torch.Tensor(val_conv_feats.reshape(val_conv_feats.shape[0], -1))
val_tense.size()

torch.Size([345, 32256])

In [217]:
test_tense = torch.Tensor(test_conv_feats.reshape(test_conv_feats.shape[0], -1))
test_tense.size()

torch.Size([1531, 32256])

In [218]:
tr_labels_tense = torch.LongTensor(np.array(tr_labels))
tr_labels_tense.size()

torch.Size([1950])

In [219]:
val_labels_tense = torch.LongTensor(np.array(val_labels))
val_labels_tense.size()

torch.Size([345])

In [220]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(classifier.parameters(), momentum=0.9, lr=0.001)

In [221]:
conv_feat_tr_ds = torch.utils.data.TensorDataset(tr_tense, tr_labels_tense)
conv_feat_tr_loader = torch.utils.data.DataLoader(conv_feat_tr_ds, shuffle=True, batch_size=32)

conv_feat_val_ds = torch.utils.data.TensorDataset(val_tense, val_labels_tense)
conv_feat_val_loader = torch.utils.data.DataLoader(conv_feat_val_ds, shuffle=False, batch_size=32)

Functions to evaluate how our model is performing.

In [222]:
from sklearn.metrics import roc_auc_score

def get_accuracy(val_loader):
    classifier.eval()
    running_loss = 0
    num_correct = 0
    num_ims = val_loader.dataset.data_tensor.size(0)
    for i, data in enumerate(val_loader):    
        ims, im_labels = data
        ims = autograd.Variable(ims, requires_grad=False)
        im_labels = autograd.Variable(im_labels.type(torch.LongTensor), requires_grad=False)
        out = classifier(ims)
        _, preds = torch.max(out, 1)
        num_correct += (preds == im_labels).sum().data[0]
    classifier.train()
    return ((num_correct * 1.0) / (num_ims * 1.0))
     

def get_auc(loader, classifier):
    classifier.eval()
    probs = torch.FloatTensor([])
    labels = torch.LongTensor([])
    for i, data in enumerate(loader):    
        ims, im_labels = data
        ims = autograd.Variable(ims, requires_grad=False)
        im_labels = autograd.Variable(im_labels.type(torch.LongTensor), requires_grad=False)
        out = classifier(ims)
        soft = nn.Softmax()
        ps  = soft(out)
        labels = torch.cat((labels,im_labels.data))
        probs = torch.cat((probs, ps.data))
    classifier.train()
    return roc_auc_score(labels.numpy(), probs.numpy()[:,1]) 

Our training loop..

In [223]:
n_epoch = 4
num_tr_ims = conv_feat_tr_ds.data_tensor.size(0) 

for epoch in range(n_epoch):
    running_loss = 0
    num_ims_completed = 0
    num_correct = 0
    for i, data in enumerate(conv_feat_tr_loader):
        optimizer.zero_grad()     
        ims, im_labels = data
        ims = autograd.Variable(ims)
        im_labels = autograd.Variable(im_labels.type(torch.LongTensor))
        out = classifier(ims)
        loss = criterion(out, im_labels)
        running_loss += loss
        loss.backward()
        optimizer.step()
        num_ims_completed += ims.size()[0]
        _, preds = torch.max(out, 1)
        num_correct += (preds == im_labels).sum().data[0]
   
    tr_acc = (num_correct * 1.0) / (num_tr_ims * 1.0)
    val_acc = get_accuracy(conv_feat_val_loader)
    print('training accuracy for epoch {}: {}'.format(epoch, tr_acc))
    print('validation accuracy for epoch {}: {}'.format(epoch, val_acc))
    print('validation AUC for epoch {}: {}'.format(epoch, get_auc(conv_feat_val_loader, classifier)))
    print('--------------------------------------')
    print('--------------------------------------')

training accuracy for epoch 0: 0.826666666667
validation accuracy for epoch 0: 0.91884057971
validation AUC for epoch 0: 0.975077656577
--------------------------------------
--------------------------------------
training accuracy for epoch 1: 0.924102564103
validation accuracy for epoch 1: 0.907246376812
validation AUC for epoch 1: 0.980603915336
--------------------------------------
--------------------------------------
training accuracy for epoch 2: 0.934871794872
validation accuracy for epoch 2: 0.939130434783
validation AUC for epoch 2: 0.982987791664
--------------------------------------
--------------------------------------
training accuracy for epoch 3: 0.949230769231
validation accuracy for epoch 3: 0.933333333333
validation AUC for epoch 3: 0.984179729827
--------------------------------------
--------------------------------------


In [224]:
torch.save(classifier.state_dict(), cwd+ '/invasive_classifier.pth')

## Make predictions on test set

In [225]:
classifier.load_state_dict(torch.load(cwd+ '/invasive_classifier.pth'))

In [226]:
% cd $data_dir

/home/ubuntu/nbs/invasive_species/data


In [227]:
samp = pd.read_csv('sample_submission.csv')

In [228]:
samp.head()

,name,invasive
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5


In [229]:
def get_preds(loader, classifier):
    classifier.eval()
    probs = torch.FloatTensor([])
    for i, data in enumerate(loader):    
        ims, _ = data
        ims = autograd.Variable(ims, requires_grad=False)
        out = classifier(ims)
        soft = nn.Softmax()
        ps  = soft(out)
        probs = torch.cat((probs, ps.data))
    return probs.numpy()[:, 1]

In [230]:
conv_feat_test_ds = torch.utils.data.TensorDataset(test_tense, torch.ones(test_tense.size()[0]))
conv_feat_test_loader = torch.utils.data.DataLoader(conv_feat_test_ds, shuffle=False, batch_size=32)

In [231]:
probs = get_preds(conv_feat_test_loader, classifier)

In [232]:
probs.shape

(1531,)

In [233]:
sub_data = np.stack((np.array(test_imgs), probs), axis=1)

In [234]:
sub_data.shape

(1531, 2)

In [235]:
sub_df = pd.DataFrame(sub_data, columns=['name', 'invasive'])

In [236]:
len(sub_df)

1531

In [237]:
sub_df.head()

,name,invasive
0,779.jpg,0.9951608181
1,1261.jpg,0.0252922773361
2,878.jpg,0.0240965969861
3,552.jpg,0.978661596775
4,400.jpg,0.368756443262


In [238]:
sub_df['name'] = sub_df.name.apply(lambda x: x.split('.')[0])

In [239]:
sub_df.head()

,name,invasive
0,779,0.9951608181
1,1261,0.0252922773361
2,878,0.0240965969861
3,552,0.978661596775
4,400,0.368756443262


In [240]:
cd $cwd

/home/ubuntu/nbs/invasive_species


In [241]:
sub_df.to_csv('invasive_clf_submission.csv', index=False)

In [242]:
from IPython.display import FileLink
FileLink('invasive_clf_submission.csv')

/home/ubuntu/nbs/invasive_species/invasive_clf_submission.csv